In [2]:
import scvi, scanpy as sc, warnings
warnings.filterwarnings("ignore")

sc.settings.set_figure_params(dpi=80, facecolor="white", frameon=False)
sc.settings.verbosity = 3

In [3]:
%%time

adata = sc.read_h5ad("Kidney_Combined_hvg5k.h5ad")
adata

CPU times: user 42.5 s, sys: 2.25 s, total: 44.7 s
Wall time: 45 s


AnnData object with n_obs × n_vars = 1975561 × 5000
    obs: 'batch', 'harmonized_celltype', 'donor', 'Sample', 'nCount_RNA', 'nFeature_RNA', 'Cell_type', 'Subcategory', 'barcode', 'celltype', 'donor_id', 'sample_uuid', 'library_uuid', 'cell_type_ontology_term_id', 'author_cell_type', 'doublet_id', 'cell_type', 'assay', 'sex', 'tissue', 'development_stage', 'Patient', 'cancer', 'CD45', 'Stage annotations', 'Grade annotations', 'file integrity', 'orig.ident', 'seurat_clusters', 'patient', 'gene_clustering', 'l', 'original_seurat_clusters', 'CN_Column', 'anno', 'cluster', 'doublet_score', 'gene_per_1kUMI', 'label', 'no_genes', 'no_genes_log10', 'pct_MT', 'sample', 'total_UMI', 'Project_ID', 'percent.mt', 'percent.ribo', 'integrated_snn_res.0.2', 'SNV_GT', 'stim', 'ident', 'Suffix', 'type', 'region', 'Sample2', 'cluster_name', 'UMAP1', 'UMAP2', 'Sample_name', 'n_genes', '_scvi_batch', '_scvi_labels', 'conditions_combined', 'modality'
    var: 'n_counts', 'highly_variable', 'highly_variabl

In [6]:
# del adata.uns["_scvi_manager_uuid"]
# del adata.uns["_scvi_uuid"]

# del adata.obsm["X_scANVI"]
# del adata.obsm["scPoli"]

In [7]:
adata.obsm

AxisArrays with keys: Harmony, Scanorama, X_bbknn, X_pca, X_scVI, X_scVI_DoubleBatch, X_tsne, X_umap

In [14]:
adata.obs["modality"].value_counts()

modality
scrna    1613391
snrna     362170
Name: count, dtype: int64

In [15]:
# https://discourse.scverse.org/t/scvi-integration-using-two-batch-keys/1758/2

bdata = adata.copy()
bdata.X = bdata.layers["counts"]

# scvi.model.SCVI.setup_anndata(adata, layer=None, batch_key=["batch", "modality"])
scvi.model.SCVI.setup_anndata(bdata, layer=None, batch_key="batch", categorical_covariate_keys=["modality"])
vae = scvi.model.SCVI(bdata, n_layers=2, n_latent=30, gene_likelihood="nb")
vae.train()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Epoch 4/4: 100%|██████████| 4/4 [06:23<00:00, 95.30s/it, v_num=1, train_loss_step=539, train_loss_epoch=545]

`Trainer.fit` stopped: `max_epochs=4` reached.


Epoch 4/4: 100%|██████████| 4/4 [06:23<00:00, 95.77s/it, v_num=1, train_loss_step=539, train_loss_epoch=545]


In [16]:
adata.obsm

AxisArrays with keys: Harmony, Scanorama, X_bbknn, X_pca, X_scVI, X_tsne, X_umap

In [17]:
adata.obsm["X_scVI_DoubleBatch"] = vae.get_latent_representation()

In [18]:
adata.write("Kidney_Combined_hvg5k.h5ad", compression="gzip")

    Save labeled subset

In [19]:
selected_dataset = [
    'Kidney_BenPublished', 
    'Kidney_BenUnpublished',
    'Kidney_Krebs',
    'Kidney_Liao',
    'Kidney_Malone',
    'Kidney_Muto', 
    'Kidney_Raji', 
    'Kidney_Wilson',
    'Kidney_Krishna',
    'Kidney_Wu'
]

adata_labeled = adata[adata.obs["harmonized_celltype"] != 'nan']
adata_labeled = adata_labeled[[_batch in selected_dataset for _batch in adata_labeled.obs["Project_ID"]]]
adata_labeled

View of AnnData object with n_obs × n_vars = 494637 × 5000
    obs: 'batch', 'harmonized_celltype', 'donor', 'Sample', 'nCount_RNA', 'nFeature_RNA', 'Cell_type', 'Subcategory', 'barcode', 'celltype', 'donor_id', 'sample_uuid', 'library_uuid', 'cell_type_ontology_term_id', 'author_cell_type', 'doublet_id', 'cell_type', 'assay', 'sex', 'tissue', 'development_stage', 'Patient', 'cancer', 'CD45', 'Stage annotations', 'Grade annotations', 'file integrity', 'orig.ident', 'seurat_clusters', 'patient', 'gene_clustering', 'l', 'original_seurat_clusters', 'CN_Column', 'anno', 'cluster', 'doublet_score', 'gene_per_1kUMI', 'label', 'no_genes', 'no_genes_log10', 'pct_MT', 'sample', 'total_UMI', 'Project_ID', 'percent.mt', 'percent.ribo', 'integrated_snn_res.0.2', 'SNV_GT', 'stim', 'ident', 'Suffix', 'type', 'region', 'Sample2', 'cluster_name', 'UMAP1', 'UMAP2', 'Sample_name', 'n_genes', '_scvi_batch', '_scvi_labels', 'conditions_combined', 'modality'
    var: 'n_counts', 'highly_variable', 'highly_

In [20]:
# https://docs.google.com/spreadsheets/d/1yS3yNlVnIlsGclgao0R9EeTPQTKF8Nx4wPupjzM3mrg/edit?gid=0#gid=0

cell_types = [
    "SULF1+ EC-AEA", "EC-AEA", "Macro", "Macro", "cycMacro", "CCD-IC-A",
    "OMCD-IC-A", "CNT-PC", "CNT-PC", "DTL", "DTL", "DTL", "EC-DVR",
    "SLC6A6+ EC-AEA", "EC-GC", "EC-PTC", "EC-PTC", "EC-AVR", "IC-B",
    "VSMC", "Pericyte", "MAST", "MC", "MD", "cycNK/T", "cycNK/T", "DC1",
    "DC2", "IC-B|CNT doub", "POD", "POD", "PEC", "PT-S1/2", "PT-S1/2_nuc",
    "PT-S3", "dPT", "dPT", "TAL", "TAL", "TAL", "ATL", "Treg", "CD4 T",
    "CD4 T", "CD4 T", "Th17", "CD8 T", "CD8 T", "CD8 T", "ILC3", "NKT",
    "Ciliated", "DCT", "EC-LYM", "OMCD-PC", "IMCD-PC", "MFAP5+aFIB",
    "aFIB", "aFIB", "aFIB", "aPT", "aPT", "CCD-PC", "CCD-PC", "DCT",
    "Neuron", "RBC", "B", "B", "PL", "TAL", "VWF+ EC-AVR", "VWF+ EC-AVR",
    "cycEC", "cycPT", "CD16+ NK", "CD56bright NK", "pDC", "cMono", "ncMono",
    "cycPapE", "PapE", "γδT", "dC-IC-A"
]


# cell_types = [ct.lower() for ct in cell_types]

adata_harmonized = adata_labeled[[celltype in cell_types for celltype in adata_labeled.obs["harmonized_celltype"]]]
adata_harmonized

View of AnnData object with n_obs × n_vars = 411838 × 5000
    obs: 'batch', 'harmonized_celltype', 'donor', 'Sample', 'nCount_RNA', 'nFeature_RNA', 'Cell_type', 'Subcategory', 'barcode', 'celltype', 'donor_id', 'sample_uuid', 'library_uuid', 'cell_type_ontology_term_id', 'author_cell_type', 'doublet_id', 'cell_type', 'assay', 'sex', 'tissue', 'development_stage', 'Patient', 'cancer', 'CD45', 'Stage annotations', 'Grade annotations', 'file integrity', 'orig.ident', 'seurat_clusters', 'patient', 'gene_clustering', 'l', 'original_seurat_clusters', 'CN_Column', 'anno', 'cluster', 'doublet_score', 'gene_per_1kUMI', 'label', 'no_genes', 'no_genes_log10', 'pct_MT', 'sample', 'total_UMI', 'Project_ID', 'percent.mt', 'percent.ribo', 'integrated_snn_res.0.2', 'SNV_GT', 'stim', 'ident', 'Suffix', 'type', 'region', 'Sample2', 'cluster_name', 'UMAP1', 'UMAP2', 'Sample_name', 'n_genes', '_scvi_batch', '_scvi_labels', 'conditions_combined', 'modality'
    var: 'n_counts', 'highly_variable', 'highly_

In [21]:
adata_harmonized.write_h5ad("kidney_hvg5k_labeled_subset.h5ad", compression="gzip")